In [1]:
import numpy as np
import queue
import copy
import matplotlib.pyplot as plt


In [2]:
# unit: hour
#即核酸检测有两个小时的检测窗口 eg. 9-11 a.m. 
#假设所有人都在这个区间进行检测👉后续可以优化
# simulation(hours)
#arrival rate (/hour) i.e. 2 per min
#service rate (/hour) 这里考虑改进前 即50s左右
#我们假设在正式检测开始之前，已经有人开始排队👉即这里的‘begin', 是一个0-30之间的随机整数
#number of server

begin = np.random.randint(0,30)
c = 3
total_time = 2
IAT_rate = 120
ST_rate = 72*c
rho = IAT_rate/ST_rate
print(begin)

20


In [3]:
# Initialize Parameters
# IAT  empty array for putting random number inter-arrival time
# St   ... for serving time rate
# AT: arrival times
# ST: serve time


qu = queue.Queue()
curr_process = None
IAT = []
ST = []
AT = []
wait_time = []
server_busy = False
list_wait = []
list_delay = []


In [4]:
# num_processes 即一共有多少个人在检测窗口里来做核酸检测

num_processes = int(np.random.poisson(IAT_rate)* total_time) 
total_people = num_processes + begin
num_processes_served = 0
print(num_processes)
print(begin)
print(total_people)

230
20
250


In [5]:
IAT_new = []

In [6]:
#Populate new Inter-Arrival-Times (IAT_new)
#即在正式检测开始之前到达的人的inter arrival time=0

for i in range(total_people):
    temp = np.random.exponential(1/IAT_rate)*60*60
    if i<begin:
        IAT_new.append(0)
        i=i+1
    else: 
        IAT_new.append(int(temp-temp%1))

print(IAT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 81, 0, 22, 63, 1, 0, 11, 84, 1, 33, 55, 23, 13, 19, 8, 32, 79, 41, 6, 7, 26, 0, 82, 32, 1, 2, 2, 26, 7, 38, 11, 23, 25, 10, 25, 40, 35, 49, 112, 9, 40, 6, 24, 2, 14, 28, 26, 19, 2, 125, 32, 31, 10, 57, 105, 36, 19, 32, 118, 9, 16, 1, 1, 10, 41, 15, 16, 1, 2, 7, 105, 6, 32, 16, 3, 27, 28, 29, 18, 30, 108, 26, 58, 21, 20, 68, 11, 78, 11, 2, 28, 0, 62, 1, 15, 20, 6, 2, 71, 26, 59, 15, 7, 81, 35, 51, 5, 26, 32, 41, 101, 50, 55, 2, 48, 38, 5, 8, 27, 12, 50, 35, 45, 21, 28, 15, 25, 39, 45, 7, 0, 16, 2, 35, 15, 7, 15, 28, 75, 11, 33, 32, 9, 56, 50, 13, 20, 34, 0, 18, 56, 55, 18, 33, 27, 5, 65, 4, 43, 44, 30, 19, 12, 28, 34, 24, 25, 13, 79, 10, 22, 70, 4, 34, 46, 18, 35, 23, 30, 49, 11, 32, 74, 11, 3, 21, 17, 8, 149, 20, 134, 1, 53, 10, 16, 20, 42, 9, 20, 13, 39, 41, 34, 9, 5, 19, 19, 51, 19, 10, 16, 0, 41, 24, 28, 57, 28, 7, 22, 22, 2, 71, 3, 37, 26, 8, 7, 9]


In [7]:
# Populate Service-Times (ST) (where ST[i]!=0)

while not len(ST) == total_people:
    temp = np.random.exponential(1/ST_rate)*60*60
    if not int(temp- temp%1)<1:
        ST.append(int(temp - temp%1))

print(ST)

[2, 28, 3, 11, 40, 55, 20, 11, 8, 1, 25, 23, 1, 3, 12, 15, 6, 34, 2, 5, 19, 11, 12, 46, 1, 11, 2, 46, 15, 60, 28, 55, 104, 23, 33, 21, 12, 19, 1, 1, 1, 52, 2, 14, 38, 1, 11, 23, 7, 7, 2, 1, 15, 9, 19, 4, 3, 17, 4, 14, 32, 48, 6, 3, 10, 31, 7, 20, 10, 39, 20, 9, 32, 2, 10, 12, 75, 11, 10, 22, 35, 8, 6, 10, 39, 10, 29, 9, 21, 2, 7, 2, 5, 9, 14, 35, 2, 9, 25, 1, 17, 2, 33, 8, 5, 1, 1, 7, 14, 19, 3, 41, 10, 51, 2, 21, 6, 12, 23, 1, 24, 3, 16, 15, 16, 16, 12, 24, 7, 29, 23, 19, 5, 67, 2, 38, 2, 7, 94, 17, 85, 1, 26, 47, 2, 10, 10, 10, 27, 15, 19, 64, 2, 12, 22, 12, 37, 18, 27, 17, 21, 22, 33, 23, 25, 15, 13, 5, 28, 61, 27, 2, 11, 34, 17, 29, 20, 90, 28, 24, 11, 9, 33, 4, 24, 5, 14, 1, 21, 1, 28, 36, 11, 11, 1, 25, 11, 10, 13, 5, 4, 8, 6, 4, 21, 1, 13, 14, 16, 12, 3, 16, 2, 33, 22, 6, 16, 3, 21, 29, 29, 27, 32, 61, 77, 6, 55, 10, 12, 5, 6, 43, 1, 5, 9, 7, 16, 6, 13, 30, 8, 4, 14, 8, 37, 23, 6, 15, 42, 5]


In [8]:
# save a copy of ST

ST_copy = copy.deepcopy(ST)

In [9]:
AT_new = []

In [10]:
# Get new Arrival-Times (AT) from IAT starting at t=0 
# 生成到达时间队列
# and initialize Waiting-Times to 0

for i in range(total_people):
    if i<begin:
        AT_new.append(0)
        i=i+1
    else:
        AT_new.append(IAT_new[i]+AT_new[i-1])
    wait_time.append(0)

print(AT_new)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 120, 120, 142, 205, 206, 206, 217, 301, 302, 335, 390, 413, 426, 445, 453, 485, 564, 605, 611, 618, 644, 644, 726, 758, 759, 761, 763, 789, 796, 834, 845, 868, 893, 903, 928, 968, 1003, 1052, 1164, 1173, 1213, 1219, 1243, 1245, 1259, 1287, 1313, 1332, 1334, 1459, 1491, 1522, 1532, 1589, 1694, 1730, 1749, 1781, 1899, 1908, 1924, 1925, 1926, 1936, 1977, 1992, 2008, 2009, 2011, 2018, 2123, 2129, 2161, 2177, 2180, 2207, 2235, 2264, 2282, 2312, 2420, 2446, 2504, 2525, 2545, 2613, 2624, 2702, 2713, 2715, 2743, 2743, 2805, 2806, 2821, 2841, 2847, 2849, 2920, 2946, 3005, 3020, 3027, 3108, 3143, 3194, 3199, 3225, 3257, 3298, 3399, 3449, 3504, 3506, 3554, 3592, 3597, 3605, 3632, 3644, 3694, 3729, 3774, 3795, 3823, 3838, 3863, 3902, 3947, 3954, 3954, 3970, 3972, 4007, 4022, 4029, 4044, 4072, 4147, 4158, 4191, 4223, 4232, 4288, 4338, 4351, 4371, 4405, 4405, 4423, 4479, 4534, 4552, 4585, 4612, 4617, 4682, 4686, 4729, 4773, 4803, 48

In [11]:
# simulate M/M/1 queue

for i in range(total_time*60*60):    
    if server_busy:
        for item in list(qu.queue):
            wait_time[item] = wait_time[item] + 1
        ST[curr_process] = ST[curr_process] - 1
        if ST[curr_process] == 0:
            server_busy = False
            num_processes_served = num_processes_served + 1

    for j in range(total_people):
        if i== AT_new[j]:
            qu.put(j)

    if not server_busy and not qu.empty():
        curr_process = qu.get()
        server_busy = True
        


In [12]:
# 总共等待时间
# 总共delay时间 = total time in the system 
# delay = waiting time + serve time
sum_wait = 0
sum_delay = 0

In [13]:
for i in range(num_processes_served):
    sum_wait = sum_wait + wait_time[i]
    sum_delay = sum_delay + wait_time[i] + ST_copy[i]
    
if num_processes_served == 0:
    list_wait.append(0)
    list_delay.append(0)
else:
    list_wait.append(sum_wait/(num_processes_served*60*60))  
    list_delay.append(sum_delay/(num_processes_served*60*60))


In [14]:
print(total_people)
print(num_processes)
print(num_processes_served)

print(num_processes_served/total_people)

250
230
250
1.0


In [15]:
#平均等待时间 in hour
print(sum_wait/(num_processes_served*60*60))
print(sum_delay/(num_processes_served*60*60))

# 平均等待时间 in minute
print(sum_wait/(num_processes_served*60))
print(sum_delay/(num_processes_served*60))

0.01767777777777778
0.022784444444444444
1.0606666666666666
1.3670666666666667
